# Data Preprocessing with SageMaker SDK (PyTorchProcessor)

This notebook demonstrates data preprocessing using the **SageMaker SDK's PyTorchProcessor** - a modern framework-based processor that simplifies preprocessing workflows.

**Key Advantages:**
- Framework-optimized for PyTorch-based workflows
- Automatic requirements.txt installation
- Built-in log streaming
- Simplified API compared to sagemaker-core

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from corelab.core.session import CoreLabSession

lab_session = CoreLabSession(
    'pytorch', 
    'customer-churn', 
    default_folder='preprocessing_sdk', 
    create_run_folder=True, 
    aws_profile='sagemaker-role'
)
lab_session.print()

/Users/machiel/Development/crystalline/sagemaker/corelab/.venv/lib/python3.13/site-packages/sagemaker_core/main/shapes.py:6645: SyntaxWarning: invalid escape sequence '\|'
  domain: The machine learning domain of the model and its components. Valid Values: COMPUTER_VISION \| NATURAL_LANGUAGE_PROCESSING \| MACHINE_LEARNING
/Users/machiel/Development/crystalline/sagemaker/corelab/.venv/lib/python3.13/site-packages/sagemaker_core/main/shapes.py:7450: SyntaxWarning: invalid escape sequence '\*'
  schedule_expression: A cron expression that describes details about the monitoring schedule. The supported cron expressions are:   If you want to set the job to start every hour, use the following:  Hourly: cron(0 \* ? \* \* \*)    If you want to start the job daily:  cron(0 [00-23] ? \* \* \*)    If you want to run the job one time, immediately, use the following keyword:  NOW    For example, the following are valid cron expressions:   Daily at noon UTC: cron(0 12 ? \* \* \*)    Daily at midnight

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/machiel/Library/Application Support/sagemaker/config.yaml


Couldn't call 'get_role' to get Role ARN from role name machiel-crystalline to get Role path.


falling back to profile: sagemaker-role
AWS region: eu-central-1
Execution role arn:aws:iam::136548476532:role/service-role/AmazonSageMaker-ExecutionRole-20250902T164316
Output bucket uri: s3://sagemaker-eu-central-1-136548476532/preprocessing_sdk/2025-10-22T09-00-07
Framework: pytorch
Project name: customer-churn


## Using PyTorchProcessor from SageMaker SDK

The PyTorchProcessor provides modern framework integration with better dependency management and simplified configuration.

In [3]:
from sagemaker.pytorch.processing import PyTorchProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

# Define data locations
data_s3_uri = f"s3://sagemaker-example-files-prod-{lab_session.region}/datasets/tabular/synthetic/churn.txt"
output_s3_uri = lab_session.jobs_output_s3_uri

print(f"📁 Data S3 URI: {data_s3_uri}")
print(f"📤 Output S3 URI: {output_s3_uri}")

📁 Data S3 URI: s3://sagemaker-example-files-prod-eu-central-1/datasets/tabular/synthetic/churn.txt
📤 Output S3 URI: s3://sagemaker-eu-central-1-136548476532/preprocessing_sdk/2025-10-22T09-00-07/customer-churn-2025-10-22T09-00-07/jobs


In [6]:
# Create PyTorchProcessor - modern framework with better dependency handling
pytorch_processor = PyTorchProcessor(
    framework_version='2.6.0',
    py_version='py312',
    instance_type="ml.m5.large", # 'ml.m5.large',
    instance_count=1,
    role=lab_session.role,
    sagemaker_session=lab_session.get_sagemaker_session(),
    volume_size_in_gb=30,
    max_runtime_in_seconds=3600,
    env={"PYTHONUNBUFFERED": "1"}
)

print("✅ PyTorchProcessor created")

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


✅ PyTorchProcessor created


In [7]:
# Run the processing job with framework features
job_name = lab_session.processing_job_name

pytorch_processor.run(
    code="preprocessing.py",         # Entry point script
    source_dir="src/",              # Directory with code and requirements.txt
    job_name=job_name,
    inputs=[
        ProcessingInput(
            source=data_s3_uri,
            destination="/opt/ml/processing/input/data"
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name="test",
            source="/opt/ml/processing/output/test",
            destination=f"{output_s3_uri}/{job_name}"
        ),
        ProcessingOutput(
            output_name="validation",
            source="/opt/ml/processing/output/validation",
            destination=f"{output_s3_uri}/{job_name}"
        ),
        ProcessingOutput(
            output_name="train",
            source="/opt/ml/processing/output/train",
            destination=f"{output_s3_uri}/{job_name}"
        )
    ],
    arguments=["--train-test-split", "0.33"],
    wait=True,  # Wait for completion
    logs=True   # Stream logs to notebook
)

print(f"✅ PyTorch Processing job completed: {job_name}")
print(f"Output location of processing job: {output_s3_uri}/{job_name}")

INFO:sagemaker.processing:Uploaded src/ to s3://sagemaker-eu-central-1-136548476532/preprocessing_sdk/2025-10-22T09-00-07/customer-churn-pytorch-processing-2025-10-22T09-00-42/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-eu-central-1-136548476532/preprocessing_sdk/2025-10-22T09-00-07/customer-churn-pytorch-processing-2025-10-22T09-00-42/source/runproc.sh
INFO:sagemaker:Creating processing-job with name customer-churn-pytorch-processing-2025-10-22T09-00-42


..............CodeArtifact repository not specified. Skipping login.
Found existing installation: typing 3.7.4.3
Uninstalling typing-3.7.4.3:
  Successfully uninstalled typing-3.7.4.3
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 MB 116.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.8/296.8 MB 47.6 MB/s  0:00:05
 

In [8]:
print(f"Output location of processing job: {output_s3_uri}/{job_name}")

Output location of processing job: s3://sagemaker-eu-central-1-136548476532/preprocessing_sdk/2025-10-22T09-00-07/customer-churn-2025-10-22T09-00-07/jobs/customer-churn-pytorch-processing-2025-10-22T09-00-42


## Key Features of PyTorchProcessor

### Advantages Over sagemaker-core ProcessingJob:

1. **Modern Framework Integration**
   - Optimized for PyTorch-based workflows
   - Uses latest PyTorch container images (2.0+)
   - Better compatibility with modern ML libraries and tools

2. **Automatic Dependency Management**
   - `source_dir` automatically uploads `requirements.txt`
   - Dependencies installed in container automatically
   - No manual container configuration needed

3. **Built-in Convenience Features**
   - `wait=True`: Automatic blocking until completion
   - `logs=True`: Stream logs directly to notebook
   - Cleaner API with less boilerplate

4. **SDK Ecosystem Integration**
   - Works seamlessly with other SDK components
   - Can be used with PipelineSession for pipelines
   - Integrates with SageMaker Experiments

### When to Use PyTorchProcessor:

✅ **Use PyTorchProcessor when:**
- Building modern ML workflows with PyTorch ecosystem
- Need automatic dependency installation
- Want simpler, higher-level API
- Building SageMaker Pipelines (next lab!)

✅ **Use sagemaker-core ProcessingJob when:**
- Need fine-grained control over all parameters
- Using custom containers
- Learning the underlying SageMaker APIs
- Building custom processing abstractions